In [321]:
from keras.layers import Dense, Flatten, Dropout, ZeroPadding3D
from keras.layers.recurrent import LSTM
from keras.models import Sequential, load_model
from glob import glob
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split, GridSearchCV, KFold

In [322]:
threshold = 40
path_cnn = '/home/fabiana/Desktop/projeto-final-src/Classifier/InceptionV3_Features/5'
proc_csv = pd.read_csv('../Vis/processedAnnotations.csv')
count = proc_csv.groupby('CM').count()['palavra']

In [323]:
def replaceClass(cg):
    quantity = count[cg]
    if (quantity <= threshold):
        return 'others'
    else: 
        return cg

In [324]:
proc_csv['classe'] = proc_csv['CM'].map(replaceClass)

In [325]:
def getWord(word):
    return re.search(r".*\/(.*)\.npy", word).group(1)

def getFiles(path):
    files = list(glob(path + "/*.npy"))
    files.sort(key=getFirstLetter)
    return files

def generateX(path):
    X = []
    files = getFiles(path)
    for file in files:
        seq = np.load(file)
        X.append(seq)
    return X

def lstm(self):
    """Build a simple LSTM network. We pass the extracted features from
    our CNN to this model predomenently."""
    # Model.
    model = Sequential()
    model.add(LSTM(2048, return_sequences=False,
                    input_shape=self.input_shape,
                    dropout=0.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(self.nb_classes, activation='softmax'))

    return model

In [326]:
# Loads features
X = generateX(path_cnn)
y = proc_csv['classe']

In [ ]:
cv = KFold(n_splits=5, shuffle=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
# Use standard fit.
# rm.model.fit(
#     X,
#     y,
#     batch_size=batch_size,
#     validation_data=(X_test, y_test),
#     verbose=1,
#     callbacks=[tb, early_stopper, csv_logger],
#     epochs=nb_epoch)